In [ ]:
#!/usr/bin/env python# coding: utf-8# In[1]:import pandas as pdimport numpy as npimport lightgbm as lgbfrom sklearn.model_selection import GroupKFoldfrom sklearn.preprocessing import LabelEncoderimport gc  # Garbage Collectorimport warningsimport syssys.path.append('/kaggle/input/ramonmartinezdataset')from ac_utils import (    clean_features,    create_features,    create_initial_datetime_features,    create_remaining_features,    reduce_mem_usage,    unify_nan_strategy,    calculate_hit_rate_at_3,)from pipeline import (    load_data,    preprocess_dataframe,    prepare_matrices,    encode_categoricals,    train_model,)warnings.filterwarnings('ignore')pd.set_option('display.max_columns', None)# Training parameters - modify as neededmodel_params = {    'objective': 'lambdarank',    'metric': 'None',    'boosting_type': 'gbdt',    'n_estimators': 1000,    'learning_rate': 0.05,    'num_leaves': 31,    'max_depth': 7,    'min_child_samples': 35,    'subsample': 0.8,    'colsample_bytree': 0.8,    'max_bin': 31,    'lambda_l1': 1.0,    'lambda_l2': 1.0,    'min_gain_to_split': 0.1,    'random_state': 42,    'n_jobs': -1,    'importance_type': 'gain',    'verbose': -1,    'seed': 42,}N_FOLDS = 3SAMPLE_FRAC = 0.1LOW_VAR_THRESH = 1CORR_THRESH = 0.95# relleno inteligente# In[2]:# In[9]:# Cell 3: Load Datatrain_df, test_df, _, test_ids_df = load_data(sample_frac=SAMPLE_FRAC)train_df_processed = preprocess_dataframe(train_df, is_train=True)test_df_processed = preprocess_dataframe(test_df, is_train=False)X, y, X_test, train_ranker_ids = prepare_matrices(train_df_processed, test_df_processed)X, X_test, cat_features = encode_categoricals(X, X_test)X, X_test, _ = clean_features(X, X_test, low_var_thresh=LOW_VAR_THRESH, corr_thresh=CORR_THRESH)categorical_features_for_encoding = cat_featuresprint(f"Final shapes -> X_train: {X.shape}, X_test: {X_test.shape}")# Cell 6: Model Trainingtest_preds_scores, feature_importances = train_model(    X,    y,    X_test,    train_ranker_ids,    categorical_features_for_encoding,    params=model_params,    n_folds=N_FOLDS,)# Display feature importance after trainingfeature_importances_display = feature_importances[['feature', 'average']].head(20)print("\nTop Feature Importances:")print(feature_importances_display)# Use the test_ids_df we saved earlier which has original Id and ranker_idsubmission_df = test_ids_df.copy()submission_df['score'] = test_preds_scores submission_df['selected'] = submission_df.groupby('ranker_id')['score'].rank(method='first', ascending=False).astype(int)# Select only required columns and ensure correct ordersubmission_df = submission_df[['Id', 'ranker_id', 'selected']]# Save submissionsubmission_df.to_parquet('submission.parquet', index=False)submission_df.to_csv('submission.csv', index=False)print("\nSubmission file 'submission.parquet' created successfully.")print(f"Submission shape: {submission_df.shape}")# Basic validation of submission# 1. All Ids from test set are presentassert len(submission_df) == len(test_ids_df), "Number of rows doesn't match test set"assert submission_df['Id'].nunique() == len(test_ids_df['Id'].unique()), "Mismatch in unique Ids"# 2. Ranks are integers and start from 1assert submission_df['selected'].min() >= 1, "Ranks should be >= 1"assert submission_df['selected'].dtype == 'int', "Ranks should be integers"# 3. Ranks are a valid permutation within each groupprint("Basic submission validation checks passed (row count, Id uniqueness, rank min value, rank dtype).")# In[ ]:# In[ ]:# In[ ]:# In[ ]:

Loading a subset of columns for train_df...
[train_df loaded] Memory usage (deep):
Index                                          145162976
Id                                             145162976
ranker_id                                     1614938108
selected                                       145162976
profileId                                      145162976
companyID                                      145162976
requestDate                                    145162976
totalPrice                                     145162976
taxes                                          145162976
legs0_departureAt                             1379048272
legs0_arrivalAt                               1379048272
legs0_duration                                1180848474
legs1_departureAt                             1150913820
legs1_arrivalAt                               1150913820
legs1_duration                                1000833781
legs0_segments0_departureFrom_airport_iata    1088722131
legs0